CNN으로 MNIST 분류하기

우선 필요한 도구들을 임포트합니다.

In [1]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

만약 GPU를 사용 가능하다면 device 값이 cuda가 되고, 아니라면 cpu가 됩니다.

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# for reproducibility
torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

학습에 사용할 파라미터를 설정합니다.

In [3]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 100

데이터로더를 사용하여 데이터를 다루기 위해서 데이터셋을 정의해줍니다.

In [4]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [5]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,batch_size=batch_size,shuffle=True,
                                          drop_last=True)

이제 클래스로 모델을 설계합니다.

In [6]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
            
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
            
        # fc layer 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        
        # fc layer 한정으로 가중치 초기화
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # 전결합층을 위해서 Flatten
        out = self.fc(out)
        # 첫번째 layer를 통과시키세요.
        # 두번째 layer를 통과시키세요.
        # fc layer를 통과시키기 전에 첫번째 차원인 배치 차원은 그대로 두고 나머지는 펼치세요.
        # fc layer를 통과시키세요.
        return out

모델을 정의합니다.

In [7]:
# instantiate CNN model
model = CNN().to(device)

비용 함수와 옵티마이저를 정의합니다.

In [8]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

총 배치의 수를 출력해보겠습니다.

In [9]:
total_batch = len(data_loader)
print(total_batch)

600


총 배치의 수는 600입니다. 그런데 배치 크기를 100으로 했으므로 결국 훈련 데이터는 총 60,000개란 의미입니다. 이제 모델을 훈련시켜보겠습니다. (시간이 꽤 오래 걸립니다.)

In [10]:
# train my model

for epoch in range(training_epochs):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        #이 부분을 채워넣으세요. # gradient를 0으로 초기화
        optimizer.zero_grad()
        hypothesis = model(X) #이 부분을 채워넣으세요. #모델을 생성해서 hypothesis를 구해보세요.
        cost = criterion(hypothesis, Y) #이 부분을 채워넣으세요. #위에서 정의한 비용함수를 사용하세요.
        cost.backward()
        optimizer.step()
        
        #이 부분을 채워넣으세요. # 비용 함수를 미분하여 gradient 계산
        #이 부분을 채워넣으세요. # W와 b를 업데이트

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Learning Finished!')

[Epoch:    1] cost = 0.225674734
[Epoch:    2] cost = 0.0631330013
[Epoch:    3] cost = 0.0463134199
[Epoch:    4] cost = 0.0374638215
[Epoch:    5] cost = 0.0314873569
[Epoch:    6] cost = 0.0262800772
[Epoch:    7] cost = 0.0217580572
[Epoch:    8] cost = 0.018268602
[Epoch:    9] cost = 0.016018346
[Epoch:   10] cost = 0.0130680576
[Epoch:   11] cost = 0.00981560536
[Epoch:   12] cost = 0.0100297388
[Epoch:   13] cost = 0.00833645649
[Epoch:   14] cost = 0.00656414125
[Epoch:   15] cost = 0.00639679609
Learning Finished!


이제 테스트를 해보겠습니다.

학습을 진행하지 않을 것이므로 torch.no_grad()

In [11]:
# Test model and check accuracy
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

C:\Users\hyejeong\anaconda3\envs\tf27_py37\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
C:\Users\hyejeong\anaconda3\envs\tf27_py37\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9866999983787537


98%의 정확도를 얻습니다. 다음 챕터에서는 층을 더 쌓아보겠습니다.